In [221]:
%run "C:\Users\dharmendrkumar\Downloads\Data Engineering\Youtube\Data_Models.ipynb"

In [222]:
from pyspark.sql.functions import count, sum, avg, col, count, to_date, count
from pyspark.sql.functions import to_date, to_timestamp, unix_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [223]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("youtube") \
    .config("spark.local.dir", r"C:\Users\dharmendrkumar\Downloads\Data Engineering\Youtube\temp") \
    .getOrCreate()

In [224]:
# Define the date and timestamp formats
date_format = 'yy.dd.MM'
timestamp_format = 'yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\''

# Read the CSV file with specified date and timestamp formats
youtube_in_original = spark.read.csv(
    r"C:\Users\dharmendrkumar\Downloads\Data Engineering\Youtube\Data\INvideos.csv",
    header=True,
    schema=youtube_india_DM,
    dateFormat=date_format,
    timestampFormat=timestamp_format
)

# Convert 'trending_date' to DateType
youtube_in_original = youtube_in_original.withColumn(
    'trending_date',
    to_date(unix_timestamp('trending_date', date_format).cast('timestamp'))
)

# Convert 'publish_time' to TimestampType
youtube_in_original = youtube_in_original.withColumn(
    'publish_time',
    to_timestamp('publish_time', timestamp_format)
)

# Find ou the top 10 trending videos

In [225]:
from pyspark.sql.functions import expr
# Replace NULL values in 'views' column with '0'
youtube_in_original = youtube_in_original.na.fill({'views': 0})

# Convert 'views' column to integer type
youtube_in_original = youtube_in_original.withColumn('views', expr('CAST(views AS INT)'))

# Sort in descending order based on 'views'
youtube_in_sorted = youtube_in_original.sort(col("views").desc())

# Show the top 10 trending videos
youtube_in_sorted.show(10)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+---------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|    views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+---------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|FlsCjmMhFmw|   2017-12-12|YouTube Rewind: T...|   YouTube Spotlight|         24|2017-12-06 17:58:51|"Rewind|""Rewind ...|125432237|2912710| 1545017|       807558|https://i.ytimg.c...|            FALSE|      

In [226]:
youtube_in_original.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = false)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [227]:
youtube_trending = youtube_in_sorted.limit(10)

In [228]:
youtube_trending.show(20)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+---------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|    views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+---------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|FlsCjmMhFmw|   2017-12-12|YouTube Rewind: T...|   YouTube Spotlight|         24|2017-12-06 17:58:51|"Rewind|""Rewind ...|125432237|2912710| 1545017|       807558|https://i.ytimg.c...|            FALSE|      

In [229]:
import os 
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [230]:
youtube_in_sorted.where(col("views") >= '10000').count()

Py4JJavaError: An error occurred while calling o1200.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 111.0 failed 1 times, most recent failure: Lost task 7.0 in stage 111.0 (TID 499) (USHYDDHARMENDR8.us.deloitte.com executor driver): java.io.FileNotFoundException: C:\Users\dharmendrkumar\AppData\Local\Temp\blockmgr-776b71ea-6ffc-4399-8fe5-c0388cb2c8b1\14\temp_shuffle_455b7eca-0436-4fe9-b218-3497e2f9b4e5 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:141)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:161)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:308)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(Byp